In [2]:
!pip install torch torchvision

  Found existing installation: Pillow 5.0.0
    Uninstalling Pillow-5.0.0:


Exception:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\shutil.py", line 544, in move
    os.rename(src, real_dst)
PermissionError: [WinError 5] Access is denied: 'c:\\programdata\\anaconda3\\lib\\site-packages\\pil' -> 'C:\\Users\\STUDEN~1.CIN\\AppData\\Local\\Temp\\pip-fxxny4ox-uninstall\\programdata\\anaconda3\\lib\\site-packages\\pil'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\pip\basecommand.py", line 215, in main
    status = self.run(options, args)
  File "C:\ProgramData\Anaconda3\lib\site-packages\pip\commands\install.py", line 342, in run
    prefix=options.prefix_path,
  File "C:\ProgramData\Anaconda3\lib\site-packages\pip\req\req_set.py", line 778, in install
    requirement.uninstall(auto_confirm=True)
  File "C:\ProgramData\Anaconda3\lib\site-packages\pip\req\req_install.py", line 754, in uninstall
    paths_to_remove.remove(auto_confi

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

# 1. Load digits dataset from sklearn
digits = load_digits()
X, y = digits.data, digits.target  # X: [n_samples, 64], y: [n_samples]

# 2. Preprocess: Normalize and split data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# 3. Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# 4. Dataset and DataLoader
train_dataset = torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = torch.utils.data.TensorDataset(X_test_tensor, y_test_tensor)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

# 5. Define simple ANN with 3 layers (input, hidden, output)
class SimpleANN(nn.Module):
    def __init__(self):
        super(SimpleANN, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Linear(128, 10)  # 10 classes (digits 0-9)
        )

    def forward(self, x):
        return self.model(x)

model = SimpleANN()

# 6. Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 7. Training loop
for epoch in range(10):  # 10 epochs
    for batch_X, batch_y in train_loader:
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print(f"Epoch [{epoch+1}/10], Loss: {loss.item():.4f}")

# 8. Evaluation
model.eval()
with torch.no_grad():
    y_pred = []
    for batch_X, _ in test_loader:
        outputs = model(batch_X)
        _, predicted = torch.max(outputs, 1)
        y_pred.extend(predicted.tolist())

accuracy = accuracy_score(y_test, y_pred)
print(f"\nTest Accuracy: {accuracy * 100:.2f}%")

Epoch [1/10], Loss: 1.1829
Epoch [2/10], Loss: 0.3708
Epoch [3/10], Loss: 0.2740
Epoch [4/10], Loss: 0.2734
Epoch [5/10], Loss: 0.2012
Epoch [6/10], Loss: 0.1725
Epoch [7/10], Loss: 0.0548
Epoch [8/10], Loss: 0.1429
Epoch [9/10], Loss: 0.0838
Epoch [10/10], Loss: 0.0289

Test Accuracy: 96.94%


In [5]:
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score
import numpy as np

# 1. Confusion matrix and classification report
conf_mat = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_mat)

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# 2. Compute probabilities for ROC AUC
model.eval()
all_probs = []
with torch.no_grad():
    for batch_X, _ in test_loader:
        outputs = model(batch_X)
        probs = torch.softmax(outputs, dim=1)  # Get probabilities
        all_probs.append(probs.cpu().numpy())

y_probs = np.vstack(all_probs)  # shape [n_samples, 10]

# 3. Compute ROC AUC for each class (one-vs-rest)
# For that, convert y_test to one-hot encoding
from sklearn.preprocessing import label_binarize

y_test_onehot = label_binarize(y_test, classes=np.arange(10))

roc_auc_scores = roc_auc_score(y_test_onehot, y_probs, average=None)

print("\nROC AUC Scores per class:")
for i, score in enumerate(roc_auc_scores):
    print(f"Class {i}: {score:.4f}")

# Optionally, average ROC AUC score
print(f"\nAverage ROC AUC (macro): {roc_auc_score(y_test_onehot, y_probs, average='macro'):.4f}")
print(f"Average ROC AUC (weighted): {roc_auc_score(y_test_onehot, y_probs, average='weighted'):.4f}")


Confusion Matrix:
[[33  0  0  0  0  0  0  0  0  0]
 [ 0 28  0  0  0  0  0  0  0  0]
 [ 0  0 33  0  0  0  0  0  0  0]
 [ 0  0  1 32  0  1  0  0  0  0]
 [ 0  0  0  0 46  0  0  0  0  0]
 [ 0  0  0  0  0 44  1  0  0  2]
 [ 1  0  0  0  0  0 34  0  0  0]
 [ 0  0  0  0  0  0  0 33  0  1]
 [ 0  1  0  0  0  1  0  0 28  0]
 [ 0  0  0  0  0  1  0  0  1 38]]

Classification Report:
             precision    recall  f1-score   support

          0       0.97      1.00      0.99        33
          1       0.97      1.00      0.98        28
          2       0.97      1.00      0.99        33
          3       1.00      0.94      0.97        34
          4       1.00      1.00      1.00        46
          5       0.94      0.94      0.94        47
          6       0.97      0.97      0.97        35
          7       1.00      0.97      0.99        34
          8       0.97      0.93      0.95        30
          9       0.93      0.95      0.94        40

avg / total       0.97      0.97      0.97